# Data Preparation of DR16Q Superset Catalogue

> The superset contains 1,440,615 observations of
> quasars, stars, and galaxies that were all targeted as
> quasars (or appeared in previous quasar catalogs).

In [ ]:
from astropy.io import fits
from astropy.modeling import models, fitting
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import minmax_scale
from spectres import spectres

import plot

In [ ]:
with fits.open("data/DR16Q_Superset_v3.fits") as hdul:
    data = hdul[1].data.byteswap().newbyteorder().copy()
    dataset = pd.DataFrame()
    for col in ["PLATE", "MJD", "FIBERID", "Z_QN", "Z_10K", "Z_CONF_10K", "Z_VI", "Z_CONF", "Z", "SOURCE_Z", "Z_PIPE", "ZWARNING", "Z_PCA", "ZWARN_PCA", "CLASS_PERSON", "Z_CONF"]:
        dataset[col] = data[col]
dataset.info()

> For objects that have a redshift in the columns `Z_VI` or `Z_10K` and a confidence (`Z_CONF` or `Z_CONF_10K`) of ≥ 2,
> `Z` records the corresponding redshift and `SOURCE_Z` is set to `VI`. Otherwise, if an object has a redshift in the
> columns `Z_DR6Q_HW` or `Z_DR7Q_SCH` these values are used (with `Z_DR6Q_HW` overriding `Z_DR7Q_SCH`) and `SOURCE_Z` is
> set to `DR6Q_HW` or `DR7QV_SCH`. As the `ZDR7Q_HW` redshifts did not formally appear in the Shen et al. (2011) paper,
> these values are not used to populate the `Z` column.
> If no other visual inspection redshift is populated then `Z_DR12Q` is used (and `SOURCE_Z` is set to `DR12QV`).
> For objects with DR12Q redshifts, only the visual inspection redshifts are recorded; DR12Q pipeline redshifts
> are not included. In the absence of any of these visual
> inspection redshifts, `Z` is populated with the automated pipeline redshift (and `SOURCE_Z` is set to `PIPE`).

In [ ]:
source_idx = dataset["SOURCE_Z"] != "PIPE"
gt_zero_idx = dataset["Z"] > 0
eq_zero_idx = dataset["Z"] == 0
source_idx.sum(), gt_zero_idx.sum(), eq_zero_idx.sum()

In [ ]:
dataset = dataset[source_idx & (gt_zero_idx | eq_zero_idx)]
dataset.head()

In [ ]:
dataset["CLASS_PERSON"].value_counts()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("z")
ax.set_ylabel("Density")
sns.kdeplot(data=dataset, x="Z", ax=ax);

## Wavelength Range

In [ ]:
with fits.open("data/specObj-dr16.fits") as hdul:
    data = hdul[1].data.byteswap().newbyteorder().copy()
    specobj = pd.DataFrame()
    for col in ["PLATE", "MJD", "FIBERID", "WAVEMIN", "WAVEMAX"]:
        specobj[col] = data[col]
specobj.head()

In [ ]:
dataset = pd.merge(dataset, specobj, on=["PLATE", "MJD", "FIBERID"], how="left")
dataset.head()

In [ ]:
dataset[["WAVEMIN", "WAVEMAX"]].describe()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Wavelength")
ax.set_ylabel("Count")
lammin = dataset["WAVEMIN"].quantile(0.999)
lammax = dataset["WAVEMAX"].quantile(0.001)
ax.axvline(lammin, color="k", linestyle="--")
ax.axvline(lammax, color="k", linestyle="-.")
sns.ecdfplot(data=dataset, x="WAVEMIN", stat="count", label="Minimal Wavelength", ax=ax)
sns.ecdfplot(data=dataset, x="WAVEMAX", stat="count", label="Maximal Wavelength", complementary=True, ax=ax)
ax.legend();

In [ ]:
lammin, lammax

In [ ]:
lam_idx = (dataset["WAVEMIN"] < lammin) & (dataset["WAVEMAX"] > lammax)
dataset.shape[0], lam_idx.sum()

In [ ]:
loglammin, loglammax = np.log10(lammin), np.log10(lammax)
loglammin, loglammax

In [ ]:
dataset = dataset[lam_idx]
dataset

In [ ]:
dataset.to_csv("data/dataset.csv", index=False)

## Preview

In [ ]:
rnd_idx = np.random.randint(dataset.shape[0])
plate, mjd, fiberid = dataset.iloc[rnd_idx][["PLATE", "MJD", "FIBERID"]]
filename = "spec-{:04d}-{}-{:04d}.fits".format(plate, mjd, fiberid)
filepath = "data/DR16Q_Superset_v3/{:04d}/".format(plate) + filename
with fits.open(filepath) as hdul:
    data = hdul[1].data
    loglam = data["loglam"]
    flux = data["flux"]

fig, ax = plt.subplots()
ax.plot(loglam, flux, label=filename)
ax.set_xlabel("Wavelength")
ax.set_ylabel("Flux")
ax.legend();

In [ ]:
loglam_idx = (loglammin <= loglam) & (loglam <= loglammax)
n_pixels = lam_idx.sum()
n_pixels

In [ ]:
fig, ax = plt.subplots()
ax.plot(loglam[loglam_idx], flux[loglam_idx], label=filename)
ax.set_xlabel("Wavelength")
ax.set_ylabel("Flux")
ax.legend();

In [ ]:
N_FEATURES = 512
EPS = 0.0005
new_loglam = np.linspace(loglammin + EPS, loglammax - EPS, N_FEATURES)
new_flux = minmax_scale(
    spectres(new_loglam, loglam[loglam_idx], flux[loglam_idx], verbose=True).astype(np.float32, copy=False).reshape(1, -1),
    feature_range=(-1, 1), axis=1, copy=False
)

In [ ]:
new_loglam.shape, new_flux.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(new_loglam, new_flux[0], label=filename)
ax.set_xlabel("Wavelength")
ax.set_ylabel("Flux")
ax.legend();

## TODO: Continuum Normalisation

1. normalize continuum
2. proper scaling

In [ ]:
flux = X[np.random.randint(ids.shape[0])]

fit = fitting.LevMarLSQFitter()    # initialize a linear fitter
line_init = models.Chebyshev1D(3)    # initialize a linear model
fitted_line = fit(line_init, plot.WAVE, flux)    # fit the data with the fitter
continuum = fitted_line(plot.WAVE)

fig, ax = plt.subplots()
plot.spectrum(ax, flux)
ax.plot(plot.WAVE, continuum)
plot.spectrum(ax, flux - continuum)

## Save Data to a HDF5 File

In [ ]:
ID_DTYPE = [("plate", np.int), ("mjd", np.int), ("fiberid", np.int)]

with h5py.File("data/dataset.hdf5", "x") as datafile:
    ids = np.zeros(len(dataset["PLATE"]), dtype=ID_DTYPE)
    ids["plate"], ids["mjd"], ids["fiberid"] = dataset["PLATE"], dataset["MJD"], dataset["FIBERID"]
    ids_dset = datafile.create_dataset("id", data=ids)
    z_dset   = datafile.create_dataset("z", data=dataset["Z"])